# 本项目暂时只能采集一个商品中的评论
## 如果需要几篇一起采集,需要解决_m_h5_tk这个关键参数的获取

In [14]:
import requests
import re
import parsel
import os
import csv
import pandas as pd
import json
import time
import hashlib
import subprocess

# TaoBao中的重要参数

## cookies

cookies = {<br>
    "sgcookie": ,<br>
    "wk_cookie2": ,<br>
    "wk_unb": ,<br>
    "_m_h5_tk": ,
    "_m_h5_tk_enc": ,<br>
}
<br>
_m_h5_tk以'_'为分割第一部分是它的token<br>第二部分是它的时间戳time(这个时间戳的作用应该是每次发请求,到一定时间后,会给你设定一个新的用,具体规则应该和后端有关系)

## params
params = {<br>
    "appKey": 商品id,<br>
    "t": 时间戳,<br>
    "sign": 加密参数,目前为止是MD5的加密方式,<br>
    "data": 商品信息<br>
}

In [17]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}
cookies = {
    "t": "e05f3fdc5ad080cfcec86db1e56b22bc",
    "thw": "xx",
    "xlly_s": "1",
    "3PcFlag": "1729839937089",
    "lgc": "tb583312559",
    "cancelledSubSites": "empty",
    "dnk": "tb583312559",
    "tracknick": "tb583312559",
    "havana_lgc2_0": "eyJoaWQiOjIyMDY1MzcyMDk4NjcsInNnIjoiZmNmY2NkNzhlNTg3YTJjYzgyOGI0ODY2MmRjY2JlZDEiLCJzaXRlIjowLCJ0b2tlbiI6IjE0UkIyd21Ja0duZHRwcGg4bmF0WENRIn0",
    "_hvn_lgc_": "0",
    "cookie3_bak": "129efd8b721e9aeb225ea4421a8cd5da",
    "cookie3_bak_exp": "1730099156770",
    "wk_cookie2": "1faa970499b30a34c963a097632d0f17",
    "wk_unb": "UUphzOfdcXH2sDxlLA%3D%3D",
    "sn": "",
    "env_bak": "FM%2BgmqK9ZWm5tV%2BjJisTgPurs7PGi4iF2LR6kanHX05Z",
    "cna": "tWFTH6FNNXMBASQMyYO/XS0z",
    "mtop_partitioned_detect": "1",
    "x5sec": "7b22733b32223a2239346234373266353637613163386232222c22617365727665723b33223a22307c434c2f7238376747454f335673387a2b2f2f2f2f2f774561447a49794d4459314d7a63794d446b344e6a63374e4443586d65546242513d3d227d",
    "_m_h5_tk": "b37c3ed36a8d57cec047aa636c5d599b_1729959133782",
    "_m_h5_tk_enc": "63ece885d3b290658508ba73907cd7ed",
    "_tb_token_": "36eb7e1a6e050",
    "_samesite_flag_": "true",
    "sgcookie": "E1001KfNH0psRfVq%2BqCBvClMUAR8qTh%2FXZ9lJvoUTz3O4tyulx7213Yzd9wKeVLfo4bIN2fldbvf51QSEUY%2BBH1xxx7Mu%2FKAae6piTH1cWoc603jaEoV7zlU5YIKFULdk1Ok",
    "havana_lgc_exp": "1761055511019",
    "unb": "2206537209867",
    "uc1": "cookie15=V32FPkk%2Fw0dUvg%3D%3D&existShop=false&pas=0&cookie21=W5iHLLyFe3xm&cookie14=UoYcComnKsSUoA%3D%3D&cookie16=V32FPkk%2FxXMk5UvIbNtImtMfJQ%3D%3D",
    "uc3": "lg2=UtASsssmOIJ0bQ%3D%3D&vt3=F8dD37r%2BverdUdaBDu8%3D&id2=UUphzOfdcXH2sDxlLA%3D%3D&nk2=F5RASpxqzEMZAew%3D",
    "csg": "91f93347",
    "cookie17": "UUphzOfdcXH2sDxlLA%3D%3D",
    "skt": "a6a68a302d8f3e07",
    "cookie2": "23beb146853c8238d64817ceaaec49d4",
    "existShop": "MTcyOTk1MTUxMQ%3D%3D",
    "uc4": "nk4=0%40FY4L5jbKwUYrE5nm82OwMCZ1NLzNdg%3D%3D&id4=0%40U2grF8wQ7DPRYS5MiAYZ%2FF9JIrGK0UrD",
    "_cc_": "W5iHLLyFfA%3D%3D",
    "_l_g_": "Ug%3D%3D",
    "sg": "977",
    "_nk_": "tb583312559",
    "cookie1": "BvLcywHzjBT%2BvP8T18nVj7dYnS4ns%2BLDKEDBQeGGauQ%3D",
    "sdkSilent": "1730037911046",
    "havana_sdkSilent": "1730037911046",
    "tfstk": "fYznaXadNkoQpNxisGuCqeqE8eStpBgSU8L-e4HPbAk_9UCQ9lczeR15O2FrE82tCWLROWEMZSwgE01IeYkue7rd6ZQYOWgSzLXAkZLfzfFuKeoE4_8ZZbu90fKaOWgW1CpPDlPBEcOfK8uzUVoZaburaHoFsVlxg48E4e-wQAMZU4oezFlZiXJeUDkFs5ksa4krgWYE3zz_79trruiKzP4itcSTtAyTC_hEx4PnItW4SEin3WkMztQuJOkoCrWOj5w0YJh_EZXiPRZzUb2wQTnQal2itJBDFYUQ9zDLtMWasVMn7yrDaUPi-viazc9hIxzQ_rgZXZ50_PwLOPVJaalTHAPQ84bVG20ELVc_yOTEElrzJDgWLtnQal2itqSPKh-VWPLS__UwV3iE1fDv3-imgZv-9dCGsngSYfGdH1fMV3iE1fDAs1xjPDls9tC..",
    "isg": "BNnZ4fjv2lTQFYcYZa6V16FZ6MWzZs0Y7tkzT_uO-4B-AviUQrfl6k9UBMZ0umVQ"
}
url = "https://h5api.m.taobao.com/h5/mtop.alibaba.review.list.for.new.pc.detail/1.0/"

params = {
    "jsv": "2.7.4",
    "appKey": "12574478",
    "t": "1729951517422",
    "sign": "53da8509123bd8c32136372ea7e57d42",
    "api": "mtop.alibaba.review.list.for.new.pc.detail",
    "v": "1.0",
    "isSec": "0",
    "ecode": "0",
    "timeout": "20000",
    "ttid": "2022@taobao_litepc_9.17.0",
    "AntiFlood": "true",
    "AntiCreep": "true",
    "dataType": "json",
    "valueType": "string",
    "type": "json",
    "data": "{\"itemId\":\"670794654965\",\"bizCode\":\"ali.china.tmall\",\"channel\":\"pc_detail\",\"pageSize\":20,\"pageNum\":1}"
}
response = requests.get(url, headers=headers, cookies=cookies, params=params)

In [18]:
response.text

'{"ret":["FAIL_SYS_USER_VALIDATE","RGV587_ERROR::SM::哎哟喂,被挤爆啦,请稍后重试"],"data":{"url":"https://h5api.m.taobao.com:443//h5/mtop.alibaba.review.list.for.new.pc.detail/1.0/_____tmd_____/punish?x5secdata=xdfde34dd5c7797a2868c20291217041dbc071bab14bbf1c4d1729951627a-717315356a1534659735abaac3en2206537209867a__bx__h5api.m.taobao.com%3A443%2Fh5%2Fmtop.alibaba.review.list.for.new.pc.detail%2F1.0&x5step=2&action=captcha&pureCaptcha="}}'

In [83]:
# 数据处理,准备sign
token = cookies['_m_h5_tk'].split('_')[0]
t = int(time.time() * 1000)
appKey = params['appKey']
data = json.loads(params['data'])
json_data = json.dumps(data, ensure_ascii=False).replace(" ", "")

In [84]:
# 通过python代码获取加密后的sign,目前淘宝用的是MD5的加密方式
sign = token + "&" + str(t) + "&" + appKey + "&" + json_data
MD5 = hashlib.md5()
MD5.update(sign.encode('utf-8'))
sign = MD5.hexdigest()

In [134]:
# 利用js文件模拟sign的生成
# 有时候不知道具体是哪个加密方式但是通过抓包知道具体哪一串代码生成的关键参数就需要用到这种方式

# Node.js脚本的路径
node_script_path = r'D:\academic\coding\PycahrmProjects\trainingField\algorithm\crawler\encrypt_params_TaoBao_sign.js'

# 要传递给Node.js脚本的参数列表
sign = token + "&" + str(t) + "&" + appKey + "&" + json_data
args = [sign]

# 构建命令行调用
command = ['node', node_script_path] + args

# 使用subprocess.run()执行命令
result = subprocess.run(command, capture_output=True, text=True)

# 获取命令行输出数据
sign = result.stdout.split('\n')[0]

In [ ]:
# 获取sign后,对需要和sign进行核对的params参数也进行更新
params['t'] = t
params['sign'] = sign
params['data'] = json_data

requests.get(url, headers=headers, cookies=cookies, params=params).text

In [135]:
sign

'245b23a3c3ae803a2bf0661ffa256076'

In [70]:
cookies['_m_h5_tk'].split('_')

['fbc25a3bd29ff854cbd313ef3141e7ec', '1729860008449']

In [15]:
# 数据如果是mtopjsonppcdetail20形式的,需要进行截取处理
raw_data = response.text
start_idx = raw_data.find('(') + 1
end_idx = raw_data.rfind(')')
if start_idx != 0 and end_idx != -1:
    json_str = raw_data[start_idx:end_idx]
data = json.loads(json_str)
review_data = data['data']['module']['reviewVOList']

In [146]:
data = response.json()
review_data = data['data']['module']['reviewVOList']
review_infos = [{'id' : review_info['id'],
                 'reviewContent' : review_info['reviewWordContent'],
                'reviewDate' : review_info['reviewDate'],
                'hasPic' : True if 'reviewPicPathList' in review_info else False} for review_info in review_data]

In [147]:
review_infos

[{'id': '1250016995571',
  'reviewContent': '很喜欢这件衣服秋天穿搭配个短袖刚刚好面料也很不错！',
  'reviewDate': '1个月前',
  'hasPic': False},
 {'id': '1250031979192',
  'reviewContent': '秋冬穿很合适，料子不错，值得购买，很喜欢格子类的衣服穿搭',
  'reviewDate': '1个月前',
  'hasPic': False},
 {'id': '1251834030908',
  'reviewContent': '160 s码也是可以接受的oversize 颜色特别好看...幽灵刺绣质感很好 面料薄薄的比较轻 跳舞穿好看 年度满意之一..',
  'reviewDate': '18天前',
  'hasPic': False},
 {'id': '1249380312513',
  'reviewContent': '立体设计的毛毛幽灵衬衫，面料摸起来一点都不扎，很舒服。',
  'reviewDate': '1个月前',
  'hasPic': False},
 {'id': '1250592396225',
  'reviewContent': '身高160体重94 s码刚刚好盖屁股 喜欢oversize的姐妹可以拍大一码哦～质量很好 适合初秋当小外套穿！',
  'reviewDate': '29天前',
  'hasPic': False},
 {'id': '1248580027457',
  'reviewContent': '料子好好 不扎，跳舞穿很帅\U0001faf0',
  'reviewDate': '1个月前',
  'hasPic': False},
 {'id': '1247532093448',
  'reviewContent': '衣服很合身，搭配不同风格都可以，很喜欢',
  'reviewDate': '2个月前',
  'hasPic': True},
 {'id': '1248957524679',
  'reviewContent': '还挺好看的这个衣服 版型蛮正 颜色也挺好看的没有色差',
  'reviewDate': '1个月前',
  'hasPic': False},

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument(r"--user-data-dir=C:\Users\11435\AppData\Local\Google\Chrome\User Data")  # 指定用户数据目录

cookies = {
    "t": "e05f3fdc5ad080cfcec86db1e56b22bc",
    "cna": "tWFTH6FNNXMBASQMyYO/XS0z",
    "thw": "xx",
    "cookie2": "129efd8b721e9aeb225ea4421a8cd5da",
    "_tb_token_": "e5d4a5e8ef131",
    "xlly_s": "1",
    "_samesite_flag_": "true",
    "3PcFlag": "1729839937089",
    "unb": "2206537209867",
    "lgc": "tb583312559",
    "cancelledSubSites": "empty",
    "cookie17": "UUphzOfdcXH2sDxlLA%3D%3D",
    "dnk": "tb583312559",
    "tracknick": "tb583312559",
    "_l_g_": "Ug%3D%3D",
    "sg": "977",
    "_nk_": "tb583312559",
    "cookie1": "BvLcywHzjBT%2BvP8T18nVj7dYnS4ns%2BLDKEDBQeGGauQ%3D",
    "sgcookie": "E100FfpBRwOZE1vQ5bMJcfAVtvAjeeR9oKxPXF%2FgjSFIkIjASigRWgL8npW1a1N4qkVStF4QrNtUci%2BWeuQkBLrvME2gue5FyWasMASgk%2F9qkSro0GsI5iJM5bCzB1gqb4N0",
    "havana_lgc2_0": "eyJoaWQiOjIyMDY1MzcyMDk4NjcsInNnIjoiZmNmY2NkNzhlNTg3YTJjYzgyOGI0ODY2MmRjY2JlZDEiLCJzaXRlIjowLCJ0b2tlbiI6IjE0UkIyd21Ja0duZHRwcGg4bmF0WENRIn0",
    "_hvn_lgc_": "0",
    "havana_lgc_exp": "1760943956770",
    "cookie3_bak": "129efd8b721e9aeb225ea4421a8cd5da",
    "cookie3_bak_exp": "1730099156770",
    "wk_cookie2": "1faa970499b30a34c963a097632d0f17",
    "wk_unb": "UUphzOfdcXH2sDxlLA%3D%3D",
    "uc1": "cookie16=VFC%2FuZ9az08KUQ56dCrZDlbNdA%3D%3D&existShop=false&cookie21=URm48syIYn73&cookie15=W5iHLLyFOGW7aA%3D%3D&cookie14=UoYcCojvCJ2Pvg%3D%3D&pas=0",
    "sn": "",
    "uc3": "id2=UUphzOfdcXH2sDxlLA%3D%3D&lg2=URm48syIIVrSKA%3D%3D&vt3=F8dD37r%2F3uOgWwn4EkI%3D&nk2=F5RASpxqzEMZAew%3D",
    "csg": "e721cd90",
    "env_bak": "FM%2BgmqK9ZWm5tV%2BjJisTgPurs7PGi4iF2LR6kanHX05Z",
    "skt": "ba238a80a71630a0",
    "existShop": "MTcyOTgzOTk1Ng%3D%3D",
    "uc4": "nk4=0%40FY4L5jbKwUYrE5nm82OwMCfAwJKQNQ%3D%3D&id4=0%40U2grF8wQ7DPRYS5MiAYZ%2FF9JI1GeEeE9",
    "_cc_": "V32FPkk%2Fhw%3D%3D",
    "havana_sdkSilent": "1729926359648",
    "sdkSilent": "1729926377502",
    "tfstk": "ftyodf2RA7lWkIdmmsk5tpIGQN1xV3MISypKJv3FgquXwvC7vDV0fy0RA_M8i244o4JRVuUmYyqQwgGdPHx7nPvpea17YpDKLNQOBOEW2vMFWH-2qkRS2c6pz4uFz574eNQOBtCy0vblWzLhRyXmjckE4Xlr3im-A3kET0rVmDopzBJz8iqqlDYyUXuyuKoEf3kELyrVmDGuFLuaL-wVPQ69QLmc_QGSZVrc1pJHB4Tt-u-9Id20i-0NKqveL-muDmiSwdfK75g-6VzVHLD30D4-kPWwnxlg1kn3rTvreSPbFYeAShGu-qcaHjYyacDorfyrG3JUcXP0nYePRCZYm4liHSCfyXHur5gsas_b-o0888c2rZ3L6oeriP7vnyG31kn3rTvz7gSygC7IMpiVv-RBObojmVKl5Y3voDzCnijD1vGrc0QOmiABObojmVIcmCMIamiRW",
    "mtop_partitioned_detect": "1",
    "_m_h5_tk": "52d87b6b8046ca1cda7f73e9957e60e3_1729921889022",
    "_m_h5_tk_enc": "dfce52fc2a66205f5c350276470212c6",
    "isg": "BNHRChfAUvw7Db8wLYZtL2nh4N1rPkWwq-xACbNmzRi2WvGs-45VgH-8_C680t3o"
}

url1 = 'http://item.taobao.com/item.htm?spm=a21n57.sem.item.5.22c63903aT97m1&priceTId=214642f117298422970536453e032c&utparam=%7B%22aplus_abtest%22%3A%22a8177acd3d38c5ce1f754c1b0b4b5ddc%22%7D&id=670794654965&xxc=ad_ztc'
url2 = 'https://item.taobao.com/item.htm?id=816200929550&priceTId=2147bfd317298407071412719e5240&spm=a21n57.sem.item.1.11003903o68NG9&utparam=%7B%22aplus_abtest%22%3A%22c8eea2892247f32636393e8ff4032ebd%22%7D&xxc=ad_ztc'
url3 = 'https://uland.taobao.com/sem/tbsearch?clk1=6ec6b58330045b278a4fc49384cd4149&keyword=%E5%A5%B3%E8%A3%85&localImgKey=&page=1&q=%E5%A5%B3%E8%A3%85&refpid=mm_2898300158_3078300397_115665800437&tab=all&upsId=6ec6b58330045b278a4fc49384cd4149'
# 指定 ChromeDriver 的路径
service = Service(executable_path=r'D:\tool\toolkit\chromedriver.exe')

# 使用 Service 创建 WebDriver 实例
driver = webdriver.Chrome(service=service, options=options)
driver.get(url1)

# for name, value in cookies.items():
#     cookie = {'name': name, 'value': value}
#     driver.add_cookie(cookie)

# driver.refresh()
driver.get_cookies()

[{'domain': 'item.taobao.com',
  'expiry': 1745467109,
  'httpOnly': False,
  'name': 'arms_uid',
  'path': '/',
  'sameSite': 'None',
  'secure': True,
  'value': '5a656b61-823e-42cc-8100-acc2c8481196'}]

In [9]:
from urllib.parse import urlparse, parse_qs
import json
import time

url_cookies_file = r'C:\Users\11435\Desktop\clutter\research\data\topicModels\TaoBao\tb_cookies.json'
params = {
    "jsv": "2.7.4",
    "appKey": "12574478",
    "t": "1729849603128",
    "sign": "847a2afc523d6cf3337fb576df14cf4c",
    "api": "mtop.alibaba.review.list.for.new.pc.detail",
    "v": "1.0",
    "isSec": "0",
    "ecode": "0",
    "timeout": "20000",
    "ttid": "2022@taobao_litepc_9.17.0",
    "AntiFlood": "true",
    "AntiCreep": "true",
    "dataType": "json",
    "valueType": "string",
    "type": "json",
    "data": "{\"itemId\":\"816200929550\",\"bizCode\":\"ali.china.tmall\",\"channel\":\"pc_detail\",\"pageSize\":20,\"pageNum\":3}"
}

with open(url_cookies_file, 'r', encoding='utf-8') as file:
    # 加载 JSON 数据
    json_data = json.load(file)

for i in range(len(json_data)):
    item_url = json_data[i]['url']
    cookies = {cookie['name']: cookie['value'] for cookie in json_data[i]['cookies']}

    # 对params里的data参数里的id进行修改
    parsed_url = urlparse(item_url)
    query_params = parse_qs(parsed_url.query)
    item_id = query_params['id']
    data = json.loads(params['data'])
    data['itemId'] = item_id
    params['data'] = json.dumps(data, ensure_ascii=False).replace(" ", "")


In [13]:
params = {
    "jsv": "2.7.4",
    "appKey": "12574478",
    "t": "1729849603128",
    "sign": "847a2afc523d6cf3337fb576df14cf4c",
    "api": "mtop.alibaba.review.list.for.new.pc.detail",
    "v": "1.0",
    "isSec": "0",
    "ecode": "0",
    "timeout": "20000",
    "ttid": "2022@taobao_litepc_9.17.0",
    "AntiFlood": "true",
    "AntiCreep": "true",
    "dataType": "json",
    "valueType": "string",
    "type": "json",
    "data": "{\"itemId\":\"816200929550\",\"bizCode\":\"ali.china.tmall\",\"channel\":\"pc_detail\",\"pageSize\":20,\"pageNum\":3}"
}
json.loads(params['data'])

{'itemId': '816200929550',
 'bizCode': 'ali.china.tmall',
 'channel': 'pc_detail',
 'pageSize': 20,
 'pageNum': 3}